In [1]:
import torch
from brevitas.quant_tensor import QuantTensor

scale = 0.0001
bit_width = 8
zero_point = 0.
int_value = torch.randint(low=- 2 ** (bit_width - 1), high=2 ** (bit_width - 1) - 1, size=(1, 2, 5, 5))
quant_value = (int_value - zero_point) * scale
quant_tensor_input = QuantTensor(
    quant_value,
    scale=torch.tensor(scale),
    zero_point=torch.tensor(zero_point),
    bit_width=torch.tensor(float(bit_width)),
    signed=True,
    training=True)
quant_tensor_input

/home/jiaming/miniconda3/envs/torch112/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


QuantTensor(value=tensor([[[[ 8.3000e-03,  1.0000e-04,  7.9000e-03, -6.5000e-03, -6.4000e-03],
          [-9.5000e-03,  2.5000e-03, -7.4000e-03, -1.1400e-02,  1.1000e-03],
          [ 1.0000e-02,  4.4000e-03, -1.1800e-02,  1.1000e-02, -9.0000e-04],
          [ 1.3000e-03, -1.2000e-02,  4.7000e-03,  7.7000e-03,  5.4000e-03],
          [ 1.1100e-02,  7.4000e-03,  3.9000e-03, -1.0900e-02,  1.1700e-02]],

         [[-6.5000e-03, -1.1900e-02, -1.0200e-02, -1.2600e-02, -7.0000e-03],
          [ 3.0000e-03,  1.0200e-02,  1.3000e-03, -5.5000e-03,  8.2000e-03],
          [-1.2600e-02, -4.7000e-03,  1.2400e-02,  1.7000e-03,  6.6000e-03],
          [ 1.2100e-02,  6.8000e-03,  1.7000e-03, -3.5000e-03, -4.3000e-03],
          [-1.5000e-03,  8.0000e-04,  3.1000e-03,  7.8000e-03,  1.0000e-04]]]]), scale=tensor(1.0000e-04), zero_point=tensor(0.), bit_width=tensor(8.), signed_t=tensor(True), training_t=tensor(True))

In [2]:
quant_tensor_input.scale

tensor(1.0000e-04)

In [11]:
import torch
from brevitas.nn import QuantIdentity
from brevitas.quant.fixed_point import Int8ActPerTensorFixedPoint
inp = torch.randn(1, 2, 5, 5)
act = QuantIdentity(act_quant=Int8ActPerTensorFixedPoint, return_quant_tensor=True)
act_out = act(inp)

import math
print(f"ACT fix point: {- math.log2(act_out.scale)}")

ACT fix point: 5.5095983415989975


In [13]:
torch.manual_seed(0)

import math
from brevitas.quant import Int8WeightPerTensorFixedPoint
from brevitas.nn import QuantLinear

quant_linear = QuantLinear(2, 4, weight_quant=Int8WeightPerTensorFixedPoint, bias=False)

print(f"Weight QuantTensor:\n {quant_linear.quant_weight()}")
print(f"Weight fix point: {- math.log2(quant_linear.quant_weight().scale)}")

Weight QuantTensor:
 QuantTensor(value=tensor([[-0.0078,  0.3828],
        [-0.5781, -0.5234],
        [-0.2734,  0.1875],
        [-0.0156,  0.5625]], grad_fn=<MulBackward0>), scale=tensor(0.0078, grad_fn=<DivBackward0>), zero_point=tensor(0.), bit_width=tensor(8.), signed_t=tensor(True), training_t=tensor(True))
Weight fix point: 7.0


In [ ]:
from brevitas.quant.solver import WeightQuantSolver, ActQuantSolver
class Int8WeightPerTensorFloat(WeightQuantSolver):
    quant_type = QuantType.INT # integer quantization
    bit_width_impl_type = BitWidthImplType.CONST # constant bit width
    float_to_int_impl_type = FloatToIntImplType.ROUND # round to nearest
    scaling_impl_type = ScalingImplType.STATS # scale based on statistics
    scaling_stats_op = StatsOp.MAX # scale statistics is the absmax value
    restrict_scaling_type = RestrictValueType.FP # scale factor is a floating point value
    scaling_per_output_channel = False # scale is per tensor
    bit_width = 8 # bit width is 8
    signed = True # quantization range is signed
    narrow_range = True # quantization range is [-127,127] rather than [-128, 127]
    zero_point_impl = ZeroZeroPoint # zero point is 0.